In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import tweepy
import csv
import datetime
import time
from textblob import TextBlob
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image
import nltk
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
#package Install
!pip3 install textblob
!pip install wordcloud
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
plt.style.use('fivethirtyeight')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/komaluntwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/komaluntwal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/komaluntwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def collect_data(dates):
    date_initial = [['2019-11-30', '2020-05-30',5000]]
    for i in range(len(date_initial)):
        print(date_initial[i][0],date_initial[i][1],date_initial[i][2])
        df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('"stress + since:"'+ date_initial[i][0] +'"until:"'+ date_initial[i][1] + '"lang:en"').get_items(), date_initial[i][2]))
    print(dates)
    print(len(dates))
    for i in range(len(dates)):
        print(dates[i][0],dates[i][1],dates[i][2])
        df=df.append(pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('"stress + since:"'+ dates[i][0] +'"until:"'+ dates[i][1] + '"lang:en"').get_items(), dates[i][2])))
        
    print(df)    
    df.to_csv("tweets102.csv", index=False)
    return df

In [4]:
date_list = [['2020-05-30','2020-11-30',10000],['2020-11-30','2021-05-30',10000],['2021-05-30','2021-10-15',5000]]
collect_data(date_list)

2019-11-30 2020-05-30 5000


KeyboardInterrupt: 

In [ ]:
read_df = pd.read_csv('tweets102.csv')

In [ ]:
read_df

In [ ]:
#Data cleaning
df_clean = read_df.drop(['media', 'retweetedTweet','quotedTweet','inReplyToTweetId','inReplyToUser','hashtags','cashtags','mentionedUsers','tcooutlinks','outlinks','sourceUrl','source', 'quoteCount', 'coordinates','renderedContent'],axis=1)
              

In [ ]:
df_clean

In [ ]:
#Data Visualisation
content_df = df_clean['content']
print(len(content_df))
content_df

In [ ]:
tweet_len = []
for contentRow in content_df:
    lengthofTweet = len(contentRow.split())
    tweet_len.append(lengthofTweet)

In [ ]:
df_clean.insert(2,"Word Length of Tweet", tweet_len)

In [ ]:
print('Average number of words used per tweet:', round(df_clean['Word Length of Tweet'].mean(),2))
print('Spread of number of words used per tweet:',round(df_clean['Word Length of Tweet'].std(),2))
print('Minimum number of number of words used per tweet:',df_clean['Word Length of Tweet'].min())
print('Maximum number of words used per tweet:',df_clean['Word Length of Tweet'].max())

In [ ]:
character_tweet_len = []
for contentRow in content_df:
    lengthofTweet = len(contentRow)
    character_tweet_len.append(lengthofTweet)

In [ ]:
df_clean.insert(2,"Character Length of Tweet", character_tweet_len)

In [ ]:
print('Average number of words used per tweet:', round(df_clean['Character Length of Tweet'].mean(),2))
print('Spread of number of words used per tweet:',round(df_clean['Character Length of Tweet'].std(),2))
print('Minimum number of number of words used per tweet:',df_clean['Character Length of Tweet'].min())
print('Maximum number of words used per tweet:',df_clean['Character Length of Tweet'].max())

In [ ]:
plt.hist(df_clean['Word Length of Tweet'], bins=50)
plt.ylabel('Count')
plt.xlabel('Word Length of Tweet');
plt.show()

In [ ]:
plt.hist(df_clean['Character Length of Tweet'], bins=50)
plt.ylabel('Count')
plt.xlabel('Character Length of Tweet');
plt.show()

In [ ]:
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
df_clean['Cleaned Content'] = df_clean['content'].apply(clean)

#Remove url
df_clean['Cleaned Content'] = df_clean['Cleaned Content'].str.replace('http\S+|www.\S+', '', case=False)


df_clean

In [ ]:
# Generate a word cloud image
stopwords = set(STOPWORDS)
mask = np.array(Image.open("Twitter.png"))
wordcloud = WordCloud(stopwords=stopwords,width = 3000, height = 2000, background_color="white", max_words=1000, mask=mask).generate(' '.join(clean_file['Cleaned Content']))
# create twitter image
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
# store to file
plt.savefig("twitter.png", format="png")
plt.show()

In [ ]:
#Tokenization
df_clean['Tokenized Content'] = df_clean.apply(lambda row: nltk.word_tokenize(row['Cleaned Content']), axis=1)

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(tokenized_sents):
    tags = pos_tag(tokenized_sents)
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

df_clean['POS tagged'] = df_clean['Tokenized Content'].apply(token_stop_pos)


#lematization
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

df_clean['Lemma'] = df_clean['POS tagged'].apply(lemmatize)
df_clean

In [ ]:
#Subjectivity
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
#Polarity to know how positive or negative the text is
def getpolarity(text):
     return TextBlob(text).sentiment.polarity
    
df_clean['Subjectivity'] = df_clean['Lemma'].apply(getsubjectivity)
df_clean['Polarity'] = df_clean['Lemma'].apply(getpolarity)    

#Compue negative, neutral and positive analysis
def analysis(points):
    if points < 0:
        return 'Negative'
    elif points == 0:
        return 'Neutral'
    else:
        return 'Positive'
df_clean['Analysis'] = df_clean['Polarity'].apply(analysis)

df_clean

In [ ]:
# Plot Subjectivity and Polarity

plt.figure(figsize=(8,6))
for i in range(0, df_clean.shape[0]):
    plt.scatter(df_clean['Polarity'][i], df_clean['Subjectivity'][i], color='violet')
    
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#Positive tweets percentage
positive_tweets = df_clean[df_clean.Analysis == 'Positive']
positive_tweets = positive_tweets['content']
print("Positive tweets Percentage: ")
round((positive_tweets.shape[0] / df_clean.shape[0])* 100, 1)

In [ ]:
#Negative tweets percentage
negative_tweets = df_clean[df_clean.Analysis == 'Negative']
negative_tweets = negative_tweets['content']
print("Negative tweets Percentage: ")
round((negative_tweets.shape[0] / df_clean.shape[0])* 100, 1)

In [ ]:
#Visualize the count

df_clean['Analysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df_clean['Analysis'].value_counts().plot(kind='bar')
plt.show()